In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dash_table
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus
import io
import oracledb

In [2]:

def create_oracle_connection1():
    try:
        # Inicializar el cliente Oracle en modo "thick"
        oracledb.init_oracle_client(lib_dir=r"C:\Users\kings\Downloads\Nueva carpeta\instantclient_23_4")
        
        # Crear la conexión
        connection = oracledb.connect(
            user="User_oper",
            password="TmLQL$Yq.1",
            dsn="10.56.1.76:1527/WNET"
        )
        
        print("Conexión exitosa.")
        return connection
    
    except oracledb.DatabaseError as e:
        # Captura y muestra cualquier error que ocurra durante la conexión
        error, = e.args
        print(f"Error al conectar: {error.message}")
        return None
    
# Uso de la función para crear una conexión
conn = create_oracle_connection1()

# Asegúrate de cerrar la conexión cuando termines
if conn:
    conn.close()

Conexión exitosa.


In [3]:
# Definición de la función de conexión
def create_oracle_connection1():
    try:
        oracledb.init_oracle_client(lib_dir=r"C:\Users\kings\Downloads\Nueva carpeta\instantclient_23_4")
        connection = oracledb.connect(
            user="User_oper",
            password="TmLQL$Yq.1",
            dsn="10.56.1.76:1527/WNET"
        )
        print("Conexión exitosa.")
        return connection
    except oracledb.DatabaseError as e:
        error, = e.args
        print(f"Error al conectar: {error.message}")
        return None

# Reemplazar fetch_data por una función de ejemplo
def fetch_data(fec_ini, fec_fin, cas, limit=None):
    try:
        conn = create_oracle_connection1()
        if conn is None:
            raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
        
                # Asegúrate de que las fechas estén en el formato correcto
        fec_ini = pd.to_datetime(fec_ini).strftime('%d/%m/%Y')
        fec_fin = pd.to_datetime(fec_fin).strftime('%d/%m/%Y')
        query = f"""

SELECT A.ATENAMBORICENASICOD AS ORIGEN,
       A.ATENAMBCENASICOD AS CENTRO,
       TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
       DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB.GRPETA1COD AS GETAREO,
       Y.PERSECNUM AS PACSECNUM,
       SR.SERVHOSCOD AS CODSER,
       SR.SERVHOSTIPPROCOD AS TIPPROF,
       'ATENME' AS ORIGEN_TIPO
  FROM CTAAM10 A
  LEFT OUTER JOIN CMAME10 X
    ON X.ORICENASICOD = A.ATENAMBORICENASICOD
   AND X.CENASICOD = A.ATENAMBCENASICOD
   AND X.ACTMEDNUM = A.ATENAMBNUM
  LEFT OUTER JOIN CMPER10 Y
    ON X.ACTMEDPACSECNUM = Y.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB
    ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR
    ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
 WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('{fec_ini}', 'DD/MM/YYYY') AND
       TO_DATE('{fec_fin}', 'DD/MM/YYYY')
   AND A.ATENAMBESTREGCOD = '1'
   AND Y.PERSECNUM IS NOT NULL
    AND A.ATENAMBCENASICOD ='{cas}'
   
UNION ALL

--DETALLE_B1_ATENNOMEDICAS:
SELECT B.ATENOMORICENASICOD AS ORIGEN,
       B.ATENOMCENASICOD AS CENTRO,
       TO_CHAR(B.ATENOMFEC, 'yyyymm') AS PERIODO,
       DECODE(Y1.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB1.GRPETA1COD AS GETAREO,
       Y1.PERSECNUM AS PACSECNUM,
       SR1.SERVHOSCOD AS CODSER,
       SR1.SERVHOSTIPPROCOD AS TIPPROF,
       'ATENOME' AS ORIGEN_TIPO
  FROM CTANM10 B
  LEFT OUTER JOIN CMAME10 X1
    ON X1.ORICENASICOD = B.ATENOMORICENASICOD
   AND X1.CENASICOD = B.ATENOMCENASICOD
   AND X1.ACTMEDNUM = B.ATENOMACTMEDNUM
  LEFT OUTER JOIN CMPER10 Y1
    ON X1.ACTMEDPACSECNUM = Y1.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB1
    ON CB1.GRPETAEDADCOD = X1.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR1
    ON SR1.SERVHOSCOD = X1.ACTMEDSERVHOSCOD
 WHERE B.ATENOMFEC BETWEEN TO_DATE('{fec_ini}', 'DD/MM/YYYY') AND
       TO_DATE('{fec_fin}', 'DD/MM/YYYY')
   AND B.ATENOMESTREGCOD = '1'
   AND Y1.PERSECNUM IS NOT NULL
    AND B.ATENOMCENASICOD ='{cas}'
  
UNION ALL

--DETALLE_B1_ATENODONTO:
SELECT CT.CITAMBORICENASICOD AS ORIGEN,
       CT.CITAMBCENASICOD AS CENTRO,
       TO_CHAR(CT.CITAMBPROCONFEC, 'YYYYMM') AS PERIODO,
       DECODE(Y2.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB2.GRPETA1COD AS GETAREO,
       Y2.PERSECNUM AS PACSECNUM,
       SR2.SERVHOSCOD AS CODSER,
       SR2.SERVHOSTIPPROCOD AS TIPPROF,
       'ATEODON' AS ORIGEN_TIPO
  FROM CTCAM10 CT
  LEFT OUTER JOIN CMAME10 X2
    ON CT.CITAMBORICENASICOD = X2.ORICENASICOD
   AND CT.CITAMBCENASICOD = X2.CENASICOD
   AND CT.CITAMBNUM = X2.ACTMEDNUM
  LEFT OUTER JOIN CMPER10 Y2
    ON X2.ACTMEDPACSECNUM = Y2.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB2
    ON CB2.GRPETAEDADCOD = X2.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR2
    ON SR2.SERVHOSCOD = X2.ACTMEDSERVHOSCOD
 WHERE CT.CITAMBPROCONFEC BETWEEN TO_DATE('{fec_ini}', 'DD/MM/YYYY') AND
       TO_DATE('{fec_fin}', 'DD/MM/YYYY')
   AND CT.CITAMBSERVHOSCOD IN ('E11', 'E12', 'E19')
   AND CT.ESTCITCOD = '4'
   AND Y2.PERSECNUM IS NOT NULL
    AND CT.CITAMBCENASICOD ='{cas}'
        """
        if limit:
            query = f"""
SELECT * FROM (
    SELECT A.ATENAMBORICENASICOD AS ORIGEN,
           A.ATENAMBCENASICOD AS CENTRO,
           TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
           DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
           CB.GRPETA1COD AS GETAREO,
           Y.PERSECNUM AS PACSECNUM,
           SR.SERVHOSCOD AS CODSER,
           SR.SERVHOSTIPPROCOD AS TIPPROF,
           'ATENME' AS ORIGEN_TIPO
      FROM CTAAM10 A
      LEFT OUTER JOIN CMAME10 X
        ON X.ORICENASICOD = A.ATENAMBORICENASICOD
       AND X.CENASICOD = A.ATENAMBCENASICOD
       AND X.ACTMEDNUM = A.ATENAMBNUM
      LEFT OUTER JOIN CMPER10 Y
        ON X.ACTMEDPACSECNUM = Y.PERSECNUM
      LEFT OUTER JOIN SGSS.CBGPE10 CB
        ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
      LEFT OUTER JOIN CMSHO10 SR
        ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
     WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('{fec_ini}', 'DD/MM/YYYY') AND
           TO_DATE('{fec_fin}', 'DD/MM/YYYY')
       AND A.ATENAMBESTREGCOD = '1'
       AND Y.PERSECNUM IS NOT NULL
        AND A.ATENAMBCENASICOD ='{cas}'
     
    UNION ALL

    --DETALLE_B1_ATENNOMEDICAS:
    SELECT B.ATENOMORICENASICOD AS ORIGEN,
           B.ATENOMCENASICOD AS CENTRO,
           TO_CHAR(B.ATENOMFEC, 'yyyymm') AS PERIODO,
           DECODE(Y1.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
           CB1.GRPETA1COD AS GETAREO,
           Y1.PERSECNUM AS PACSECNUM,
           SR1.SERVHOSCOD AS CODSER,
           SR1.SERVHOSTIPPROCOD AS TIPPROF,
           'ATENOME' AS ORIGEN_TIPO
      FROM CTANM10 B
      LEFT OUTER JOIN CMAME10 X1
        ON X1.ORICENASICOD = B.ATENOMORICENASICOD
       AND X1.CENASICOD = B.ATENOMCENASICOD
       AND X1.ACTMEDNUM = B.ATENOMACTMEDNUM
      LEFT OUTER JOIN CMPER10 Y1
        ON X1.ACTMEDPACSECNUM = Y1.PERSECNUM
      LEFT OUTER JOIN SGSS.CBGPE10 CB1
        ON CB1.GRPETAEDADCOD = X1.ACTMEDEDADATEN
      LEFT OUTER JOIN CMSHO10 SR1
        ON SR1.SERVHOSCOD = X1.ACTMEDSERVHOSCOD
     WHERE B.ATENOMFEC BETWEEN TO_DATE('{fec_ini}', 'DD/MM/YYYY') AND
           TO_DATE('{fec_fin}', 'DD/MM/YYYY')
       AND B.ATENOMESTREGCOD = '1'
       AND Y1.PERSECNUM IS NOT NULL
        AND B.ATENOMCENASICOD ='{cas}'
    
    UNION ALL

    --DETALLE_B1_ATENODONTO:
    SELECT CT.CITAMBORICENASICOD AS ORIGEN,
           CT.CITAMBCENASICOD AS CENTRO,
           TO_CHAR(CT.CITAMBPROCONFEC, 'YYYYMM') AS PERIODO,
           DECODE(Y2.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
           CB2.GRPETA1COD AS GETAREO,
           Y2.PERSECNUM AS PACSECNUM,
           SR2.SERVHOSCOD AS CODSER,
           SR2.SERVHOSTIPPROCOD AS TIPPROF,
           'ATEODON' AS ORIGEN_TIPO
      FROM CTCAM10 CT
      LEFT OUTER JOIN CMAME10 X2
        ON CT.CITAMBORICENASICOD = X2.ORICENASICOD
       AND CT.CITAMBCENASICOD = X2.CENASICOD
       AND CT.CITAMBNUM = X2.ACTMEDNUM
      LEFT OUTER JOIN CMPER10 Y2
        ON X2.ACTMEDPACSECNUM = Y2.PERSECNUM
      LEFT OUTER JOIN SGSS.CBGPE10 CB2
        ON CB2.GRPETAEDADCOD = X2.ACTMEDEDADATEN
      LEFT OUTER JOIN CMSHO10 SR2
        ON SR2.SERVHOSCOD = X2.ACTMEDSERVHOSCOD
     WHERE CT.CITAMBPROCONFEC BETWEEN TO_DATE('{fec_ini}', 'DD/MM/YYYY') AND
           TO_DATE('{fec_fin}', 'DD/MM/YYYY')
       AND CT.CITAMBSERVHOSCOD IN ('E11', 'E12', 'E19')
       AND CT.ESTCITCOD = '4'
       AND Y2.PERSECNUM IS NOT NULL
        AND CT.CITAMBCENASICOD ='{cas}'
) WHERE ROWNUM <= {limit}
        """

        print("Consulta SQL generada:")
        print(query)  # Agrega esto para depurar la consulta SQL

        df = pd.read_sql(query, conn)
        print(f"Cantidad de filas obtenidas: {len(df)}")  # Agrega esto para ver cuántas filas se obtienen
    except Exception as e:
        print(f"Error: {e}")
        df = pd.DataFrame()
    finally:
        if conn:
            conn.close()  # Cerrar la conexión al final
    return df



In [4]:
external_stylesheets = [dbc.themes.BOOTSTRAP,
    'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css',  # Bootstrap CSS
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css']  # Bootstrap Icons
# Crear la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    # Título
    dbc.Row([
        html.Div(style={'height': '12px'}),
        html.H1("SUSALUD - TABLA B1", style={'color': '#0064AF', 'fontSize': '28px', 'textAlign': 'Left', 'font-weight': 'bold', 'fontFamily': 'Calibri'}),
        html.H2("Fuente: ESSI. V.1.0.0", style={'color': '#0064AF', 'fontSize': '12px'}),
    ]),
    html.Hr(style={'border': '2px solid #0064AF'}),
    dbc.Row([
        # Date Picker for Start Date
            dbc.Col(
                [
                    html.Label("Fecha inicio", style={'font-weight': 'bold', 'fontFamily': 'Calibri', 'color': '#0064AF'}),
                    dcc.DatePickerSingle(
                        id='start-date-picker',
                        display_format='DD/MM/YYYY',
                        className='date-picker-custom px-0 mx-0'
                    )
                ],
                className='px-0 ml-2',
                width=3, md=4, lg=1
            ),
        # Date Picker for End Date
        dbc.Col(
            [
                html.Label("Fecha fin", style={'font-weight': 'bold', 'fontFamily': 'Calibri', 'color': '#0064AF'}),
                dcc.DatePickerSingle(
                id='end-date-picker',
                display_format='DD/MM/YYYY',
                className='date-picker-custom px-0 mx-0'
            ),],

            className='px-0 mx-0',
            width=3, md=4, lg=1
        ),
        dbc.Col(
        [
            html.Label("Centro asistencial", style={'font-weight': 'bold', 'fontFamily': 'Calibri', 'color': '#0064AF'}),    
            dcc.Input(
                id='cas',
                type='text',
                placeholder='Código de centro asistencial',
                className='px-0 mx-0',
                style={'height': '45px', 'width': '100%', 'border': '0.5px solid #606060'}
            ),],

            className='px-0 mx-0',
            width=2, md=3, lg=2
        ),
        dbc.Col(
            dbc.Button(
                html.I(className="fas fa-search"),
                id='submit-val',
                style={'background-color': '#0064AF', 'border-color': '#0064AF', 'color': 'white', 'width': '50%', 'height': '45px', 'fontSize': '20px','margin-top':'33px'},

            ),
            className='mt- 200',
            width=1, md=1, lg=1
        ),
        dbc.Col(
            dcc.Loading(
                id="loading-download",
                type="default",
                children=html.Div([
                    dbc.Button(
                        'Descargar datos',
                        id='download-btn',
                        n_clicks=0,
                        style={'background-color': '#0064AF', 'border-color': '#0064AF', 'color': 'white', 'width': '100%', 'height': '45px','margin-top':'33px'}
                    ),
                    dcc.Download(id="download-dataframe-csv")
                ]), style={'margin-top':'33px'}

            ),
            width=2, md=3, lg=2
        ),
    ], className="mb-4"),

    # Salida de datos con dbc.Spinner
    dbc.Spinner(
        id="loading-output",
        size="md",
        color="primary",
        type="border",
        fullscreen=False,
        delay_show=250,
        children=html.Div(id='output-data-table')
    ),
], fluid=True)

@app.callback(
    Output('output-data-table', 'children'),
    Input('submit-val', 'n_clicks'),
    State('start-date-picker', 'date'),
    State('end-date-picker', 'date'),
    State('cas', 'value')
)
def update_output(n_clicks, start_date, end_date, cas):
    if n_clicks is None:
        return html.Div("Ingrese el rango de fecha y el centro asistencial",style={'color':'#606060'})
    
    if n_clicks > 0:
        try:
            if not start_date or not end_date or not cas:
                return html.Div("Por favor, complete todos los campos.")
            
            df = fetch_data(start_date, end_date, cas, limit=50)

            if df.empty:
                return html.Div("No se encontraron datos para los criterios proporcionados.")
            
            return html.Div([
                html.H4("Data de muestra:", style={'color': '#0064AF', 'width': '100%', 'height': '45px', 'fontSize':'20px'}),
                dash_table.DataTable(
                    id='data-table',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict('records'),
                    style_table={'overflowX': 'auto'},
                    page_size=15,
                    style_cell={
                        'textAlign': 'left',
                        'fontFamily': 'Calibri',
                        'padding': '5px',
                        'height': 'auto',
                        'maxWidth': '120px',
                        'whiteSpace': 'normal',
                        'color': '#606060',
                        'fontSize': '14px'
                    },
                    style_header={
                        'backgroundColor': '#0064AF',
                        'fontWeight': 'bold',
                        'color': 'white'
                    },
                    style_cell_conditional=[
                        {'if': {'column_id': 'ORIGEN'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '150px', 'textAlign': 'center'},
                        {'if': {'column_id': 'CENTRO'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '200px', 'textAlign': 'center'},
                        {'if': {'column_id': 'PERIODO'}, 'minWidth': '130px', 'width': '130px', 'maxWidth': '250px'},
                        {'if': {'column_id': 'SEXO'}, 'minWidth': '120px', 'width': '120px', 'maxWidth': '150px', 'textAlign': 'center'},
                        {'if': {'column_id': 'GETAREO'}, 'minWidth': '230px', 'width': '230px', 'maxWidth': '250px'},
                        {'if': {'column_id': 'PACSECNUM'}, 'minWidth': '120px', 'width': '120px', 'maxWidth': '150px', 'textAlign': 'center'},
                        {'if': {'column_id': 'CODSER'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '150px'},
                        {'if': {'column_id': 'TIPPROF'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '100px'},
                        {'if': {'column_id': 'ORIGEN_TIPO'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '100px', 'textAlign': 'center'},
                    ],
                    style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(244, 250, 253)'}],
                )
            ])
        except Exception as e:
            return html.Div(f'Error: {e}')
        
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-btn", "n_clicks"),
    State('start-date-picker', 'date'),
    State('end-date-picker', 'date'),
    State('cas', 'value'),
    prevent_initial_call=True
)
def download_csv(n_clicks, start_date, end_date, cas):
    if n_clicks > 0:
        try:
            if not start_date or not end_date or not cas:
                return dcc.send_data_frame(pd.DataFrame().to_csv, filename="data_empty.csv")
            
            import time
            time.sleep(2)

            df_complete = fetch_data(start_date, end_date, cas, None)
            return dcc.send_data_frame(df_complete.to_csv, filename="data_complete.csv")
        except Exception as e:
            return dcc.send_data_frame(pd.DataFrame().to_csv, filename="data_empty.csv")

if __name__ == '__main__':
    app.run_server(debug=True)

Conexión exitosa.
Consulta SQL generada:

SELECT * FROM (
    SELECT A.ATENAMBORICENASICOD AS ORIGEN,
           A.ATENAMBCENASICOD AS CENTRO,
           TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
           DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
           CB.GRPETA1COD AS GETAREO,
           Y.PERSECNUM AS PACSECNUM,
           SR.SERVHOSCOD AS CODSER,
           SR.SERVHOSTIPPROCOD AS TIPPROF,
           'ATENME' AS ORIGEN_TIPO
      FROM CTAAM10 A
      LEFT OUTER JOIN CMAME10 X
        ON X.ORICENASICOD = A.ATENAMBORICENASICOD
       AND X.CENASICOD = A.ATENAMBCENASICOD
       AND X.ACTMEDNUM = A.ATENAMBNUM
      LEFT OUTER JOIN CMPER10 Y
        ON X.ACTMEDPACSECNUM = Y.PERSECNUM
      LEFT OUTER JOIN SGSS.CBGPE10 CB
        ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
      LEFT OUTER JOIN CMSHO10 SR
        ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
     WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
           TO_DATE('02/08/2024', 'DD/MM/YY

C:\Users\kings\AppData\Local\Temp\ipykernel_9588\3192133918.py:207: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Cantidad de filas obtenidas: 50
Conexión exitosa.
Consulta SQL generada:


SELECT A.ATENAMBORICENASICOD AS ORIGEN,
       A.ATENAMBCENASICOD AS CENTRO,
       TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
       DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
       CB.GRPETA1COD AS GETAREO,
       Y.PERSECNUM AS PACSECNUM,
       SR.SERVHOSCOD AS CODSER,
       SR.SERVHOSTIPPROCOD AS TIPPROF,
       'ATENME' AS ORIGEN_TIPO
  FROM CTAAM10 A
  LEFT OUTER JOIN CMAME10 X
    ON X.ORICENASICOD = A.ATENAMBORICENASICOD
   AND X.CENASICOD = A.ATENAMBCENASICOD
   AND X.ACTMEDNUM = A.ATENAMBNUM
  LEFT OUTER JOIN CMPER10 Y
    ON X.ACTMEDPACSECNUM = Y.PERSECNUM
  LEFT OUTER JOIN SGSS.CBGPE10 CB
    ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
  LEFT OUTER JOIN CMSHO10 SR
    ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
 WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
       TO_DATE('02/08/2024', 'DD/MM/YYYY')
   AND A.ATENAMBESTREGCOD = '1'
   AND Y.PERSECNUM IS NOT NULL
   

C:\Users\kings\AppData\Local\Temp\ipykernel_9588\3192133918.py:207: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Cantidad de filas obtenidas: 5636
Conexión exitosa.
Consulta SQL generada:

SELECT * FROM (
    SELECT A.ATENAMBORICENASICOD AS ORIGEN,
           A.ATENAMBCENASICOD AS CENTRO,
           TO_CHAR(A.ATENAMBATENFEC, 'yyyymm') AS PERIODO,
           DECODE(Y.PERSEXOCOD, '0', '2', '1', '1') AS SEXO,
           CB.GRPETA1COD AS GETAREO,
           Y.PERSECNUM AS PACSECNUM,
           SR.SERVHOSCOD AS CODSER,
           SR.SERVHOSTIPPROCOD AS TIPPROF,
           'ATENME' AS ORIGEN_TIPO
      FROM CTAAM10 A
      LEFT OUTER JOIN CMAME10 X
        ON X.ORICENASICOD = A.ATENAMBORICENASICOD
       AND X.CENASICOD = A.ATENAMBCENASICOD
       AND X.ACTMEDNUM = A.ATENAMBNUM
      LEFT OUTER JOIN CMPER10 Y
        ON X.ACTMEDPACSECNUM = Y.PERSECNUM
      LEFT OUTER JOIN SGSS.CBGPE10 CB
        ON CB.GRPETAEDADCOD = X.ACTMEDEDADATEN
      LEFT OUTER JOIN CMSHO10 SR
        ON SR.SERVHOSCOD = X.ACTMEDSERVHOSCOD
     WHERE TRUNC(A.ATENAMBATENFEC) BETWEEN TO_DATE('01/08/2024', 'DD/MM/YYYY') AND
        